### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR(pair-wise loss) на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP(list-wise loss) на implicit данных


In [ ]:
!pip install implicit lightfm faiss
!apt-get install libopenblas-dev
!apt-get install libomp-dev

     |████████████████████████████████| 1.1MB 18.6MB/s 
     |████████████████████████████████| 317kB 49.2MB/s 
     |████████████████████████████████| 4.7MB 51.9MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp37-cp37m-linux_x86_64.whl size=3406340 sha256=1042d384e9ef38c412cbd46f25e0bff1026ad84f478e12b08bf7ce898f684ed6
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705338 sha256=91d55dce9ea1027acbfdce171088786ca274498659f5dbdb6b9e7fe34262f2f6
  Stored in directory: /root/.cache/pip/wheels/c6/64/d4/673c7277f71ac4c5ad4835b94708c01b653ef2d3aa78ef20aa
Successfully built implicit lightfm
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libopenblas-dev is already the newest version (0.2.20+ds-4).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
Reading package lists... Done
B

In [ ]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

from lightfm.datasets import fetch_movielens

In [ ]:
!wget --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-1m.zip
!mkdir RecSysHSE
!unzip ml-1m.zip -d RecSysHSE/

--2021-03-25 15:32:31--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  26.1MB/s    in 0.2s    

2021-03-25 15:32:31 (26.1 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: RecSysHSE/ml-1m/
  inflating: RecSysHSE/ml-1m/movies.dat  
  inflating: RecSysHSE/ml-1m/ratings.dat  
  inflating: RecSysHSE/ml-1m/README  
  inflating: RecSysHSE/ml-1m/users.dat  


В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [ ]:
ratings = pd.read_csv('RecSysHSE/ml-1m/ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [ ]:
movie_info = pd.read_csv('RecSysHSE/ml-1m/movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')

Explicit данные

In [ ]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [ ]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [ ]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [ ]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [ ]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [ ]:
model.fit(user_item_t_csr)

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [ ]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [ ]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '584    Aladdin (1992)',
 '360    Lion King, The (1994)',
 '2315    Babe: Pig in the City (1998)',
 '1838    Mulan (1998)',
 '1526    Hercules (1997)',
 '2618    Tarzan (1999)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [ ]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [ ]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [ ]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [ ]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '1182    Aliens (1986)',
 '2502    Matrix, The (1999)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '3402    Close Encounters of the Third Kind (1977)',
 '847    Godfather, The (1972)',
 '2460    Planet of the Apes (1968)',
 '1179    Princess Bride, The (1987)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [ ]:
from scipy.sparse.linalg import spsolve
from scipy import sparse
from tqdm.auto import tqdm
from sklearn.neighbors import KNeighborsClassifier

class ALS():
  def __init__(self, data, r_lambda = 40, nf = 200, alpha = 40):
    self.data = data
    self.r_lambda = r_lambda
    self.nf = nf
    self.alpha = alpha

    self.nu = data.shape[0]
    self.ni = data.shape[1]

    self.X = sparse.csr_matrix(np.random.rand(self.nu, self.nf) * 0.01) #user latent matrix
    self.Y = sparse.csr_matrix(np.random.rand(self.ni, self.nf) * 0.01) #item latent matrix

    self.C = alpha * self.data #confidence matrix

  def optimize_user(self):
      yT = self.Y.T
      yT_Cu_y = yT.dot(self.Y)

      for u in tqdm(range(self.nu), leave=False):
          c = self.C[u, :].toarray() 
          p = c.copy()
          p[p > 0] = 1

          d = sparse.diags(c, [0])
          conf = d + sparse.eye(self.Y.shape[0])
          
          yT_Cu_pu = yT.dot(conf).dot(p.T)
          lI = yT.dot(d).dot(self.Y)
          self.X[u] = spsolve(yT_Cu_y + lI, yT_Cu_pu)

  def optimize_item(self):
      xT = self.X.T
      xT_Ci_x = xT.dot(self.X)

      for i in tqdm(range(self.ni), leave=False):
          c = self.C[:, i].T.toarray() 
          p = c.copy()
          p[p > 0] = 1

          d = sparse.diags(c, [0])
          conf = d + sparse.eye(self.X.shape[0])

          xT_Ci_pi = xT.dot(conf).dot(p.T)
          lI = xT.dot(d).dot(self.X)
          self.Y[i] = spsolve(xT_Ci_x + lI, xT_Ci_pi)
  
  def fit_KNN(self):
    self.nbrs = NearestNeighbors(n_neighbors=100).fit(self.Y)

  def similar_items(self, item_id):
    distances, indices = self.nbrs.kneighbors(self.Y[item_id])
    distances = distances[0][:10]
    indices = indices[0][:10]
    return zip(list(indices), list(distances))
  
  def recommend(self, user_id, user_item_csr):
    distances, indices = nbrs.kneighbors(self.X[user_id])
    distances = distances[0]
    indices = indices[0]
    recommendation = []
    user_data = user_item_csr[user_id, :].toarray().flatten()
    for i, d in zip(list(indices), list(distances)):
        if user_data[i] == 0: recommendation.append((i, d))
    return recommendation[:10]

In [ ]:
als = ALS(user_item_csr)

for i in range(1):
      als.optimize_user()
      als.optimize_item()


In [ ]:
als.fit_KNN()

In [ ]:
get_similars(1, als)

['0    Toy Story (1995)',
 '2669    Crimes of the Heart (1986)',
 '990    Extreme Measures (1996)',
 '772    Fan, The (1996)',
 '1344    Ridicule (1996)',
 '2108    Family Plot (1976)',
 '133    Down Periscope (1996)',
 '175    Lord of Illusions (1995)',
 '2079    House (1986)',
 '2111    Torn Curtain (1966)']

In [ ]:
get_recommendations(4, als)

['3798    Naked Gun: From the Files of Police Squad!, Th...',
 '3466    American Psycho (2000)',
 '2385    Fly, The (1958)',
 '110    Rumble in the Bronx (1995)',
 '2879    From Russia with Love (1963)',
 '3846    Remember the Titans (2000)',
 '3549    Small Time Crooks (2000)',
 '3558    Carnival of Souls (1962)',
 '3585    Guns of Navarone, The (1961)',
 '341    Adventures of Priscilla, Queen of the Desert, ...']

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных